# The `next_tool` and `manual_call`

## Overview 

If you have worked with function calls before, you know that often the model does not call the function you want to call. This poses a knonw problem of AI Agents, because often want a particular flow to be respected. 

In this notebook, we will see how to use the `next_tool` and `manual_call` to control the flow of the AI Agent, make it more reliable and predictable.

#### next_tool

The `next_tool` is a parameter that can be used with the `function_tool` or  `agent_tool` to specify the next tool to be called. Behind the scenes, the `next_tool` is just setting the completion parameter `tool_choice` to the name of the next tool to be called. 

To use it, you just need to pass the name of of the method you want to call next as the value of the `next_tool` parameter. Note that the method must be decored as a tool and must be in the same class as the current tool.


#### manual_call

The `manual_call` is a parameter that can be used together with the `next_tool` to specify that the next tool call will be inserted "manually" in the message history. By manually, we mean that the next tool call will not be done by a new model call, but inserted directly in the message history. This is usefull in the situations where the output of the current tool can be directly used as the input of the next tool, without the need of a new model call.

To use it, you need to pass a callable as the value of the `manual_call` parameter, that will receive the output of the current tool and return a dictionay that should match the input of the next tool. In this way you can add some processing to the output of the current tool before passing it to the next tool.

---

## Let's see how to use these parameters in practice.


In the example bellow we have a simple agent that has two tools, one to generate a random topic and another that genrate a joken, given a topic.

To show that the `next_tool` is working, we will force the agent to call the `generate_joke` tool after a random topic is generated.

In [1]:
from agente.core.base import BaseAgent,BaseTaskAgent
from agente.core.decorators import function_tool,agent_tool
import random
from dotenv import load_dotenv
load_dotenv()

True

In [2]:

class JokeTeller(BaseTaskAgent):
    agent_name: str = "JokeTeller"
    system_prompt:str = "Your task is to write a funny joke."
    completion_kwargs: dict = {
        "model": "gpt-4o-mini",
        "stream": False,
    }

    @function_tool
    def complete_task(self,joke:str):
        """To be used as a tool to complete the task.

        Args:
            joke: The joke to return.
        """
        return joke



class MainAgent(BaseAgent):
    agent_name: str = "main_agent"
    
    @function_tool(next_tool = "get_joke") # To make sure the agent calls the get_joke tool we add the next_tool argument to force it.
    def random_topic(self):
        """Tool to get a random topic.
        """
        topics = ["programming","science","animals","food","sports"]
        topic = random.choice(topics)

        return topic


    @agent_tool()
    def get_joke(self,joke_topic:str):
        """Tool to get a joke.

        Args:
            joke_topic: The topic of the joke.
        """

        joke_agent = JokeTeller()
        joke_agent.add_message(role = "user", content = "Tell me a joke about " + joke_topic)
        return joke_agent

In [3]:
example_agent = MainAgent()
example_agent.add_message(role = "user", content = "Generate a random topic.")
responses = [r async for r in example_agent.run()]

Executing agent: main_agent
Executing tool: random_topic from agent main_agent
Executing agent: main_agent
Executing agent: JokeTeller
Executing tool: complete_task from agent JokeTeller
Executing agent: main_agent


In [4]:
print(example_agent.conv_history.messages[-1].content)

The random topic is "science." Here's a science joke for you:

Why can't you trust an atom? Because they make up everything!


In [5]:
example_agent.conv_history.messages

[Message(role='user', agent_name='main_agent', content='Generate a random topic.', tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='assistant', agent_name='main_agent', content=None, tool_calls=[{'index': 0, 'function': {'arguments': '{}', 'name': 'random_topic'}, 'id': 'call_8rpvzweXykUzvjX1Z4G7wOj0', 'type': 'function'}], tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='tool', agent_name='main_agent', content='"science"', tool_calls=None, tool_call_id='call_8rpvzweXykUzvjX1Z4G7wOj0', tool_name='random_topic', hidden=False, id=None, usage=None),
 Message(role='assistant', agent_name='main_agent', content=None, tool_calls=[{'index': 0, 'function': {'arguments': '{"joke_topic":"science"}', 'name': 'get_joke'}, 'id': 'call_3iH7Zle7DY0XgV107U81vsc4', 'type': 'function'}], tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='tool', agent_name='main_agent', content='"W

An inspection to the full message history will show that the `get_joke` tool was called after the `random_topic` tool.

In [6]:
for m in example_agent.conv_history.messages:
    if m.tool_calls:
        print(m.tool_calls)

[{'index': 0, 'function': {'arguments': '{}', 'name': 'random_topic'}, 'id': 'call_8rpvzweXykUzvjX1Z4G7wOj0', 'type': 'function'}]
[{'index': 0, 'function': {'arguments': '{"joke_topic":"science"}', 'name': 'get_joke'}, 'id': 'call_3iH7Zle7DY0XgV107U81vsc4', 'type': 'function'}]


And in total we add 3 model calls:

In [7]:
len(example_agent.log_calls)

3

Now, let's see how to use the `manual_call` parameter. 

We just need to add a function that will pass the generated topic to the `get_joke` tool. We will use a simple lambda function to do that.

In [8]:
class MainAgent(BaseAgent):
    agent_name: str = "main_agent"
    
    @function_tool(next_tool = "get_joke", manual_call= lambda x: {'joke_topic':x})
    def random_topic(self):
        """Tool to get a random topic.
        """
        topics = ["programming","science","animals","food","sports"]
        topic = random.choice(topics)

        return topic


    @agent_tool()
    def get_joke(self,joke_topic:str):
        """Tool to get a joke.

        Args:
            joke_topic: The topic of the joke.
        """

        joke_agent = JokeTeller()
        joke_agent.add_message(role = "user", content = "Tell me a joke about " + joke_topic)
        return joke_agent

In [9]:
example_agent = MainAgent()
example_agent.add_message(role = "user", content = "Generate a random topic.")
responses = [r async for r in example_agent.run()]

Executing agent: main_agent
Executing tool: random_topic from agent main_agent
Executing agent: JokeTeller
Executing tool: complete_task from agent JokeTeller
Executing agent: main_agent


In [10]:
example_agent.conv_history.messages

[Message(role='user', agent_name='main_agent', content='Generate a random topic.', tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='assistant', agent_name='main_agent', content=None, tool_calls=[{'index': 0, 'function': {'arguments': '{}', 'name': 'random_topic'}, 'id': 'call_c7EyI8RGJjNzSHUCSSs5EyrW', 'type': 'function'}], tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='tool', agent_name='main_agent', content='"animals"', tool_calls=None, tool_call_id='call_c7EyI8RGJjNzSHUCSSs5EyrW', tool_name='random_topic', hidden=False, id=None, usage=None),
 Message(role='assistant', agent_name='main_agent', content=None, tool_calls=[{'id': 'man_EeLk12aazyCEPmK3VogtUxQX', 'function': {'name': 'get_joke', 'arguments': '{"joke_topic": "animals"}'}, 'type': 'function'}], tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='tool', agent_name='main_agent', content='"Why don\'t sc

In [11]:
for m in example_agent.conv_history.messages:
    if m.tool_calls:
        print(m.tool_calls)

[{'index': 0, 'function': {'arguments': '{}', 'name': 'random_topic'}, 'id': 'call_c7EyI8RGJjNzSHUCSSs5EyrW', 'type': 'function'}]
[{'id': 'man_EeLk12aazyCEPmK3VogtUxQX', 'function': {'name': 'get_joke', 'arguments': '{"joke_topic": "animals"}'}, 'type': 'function'}]


The manual call has the prefix "man_" to indicate that the tool call was inserted manually in the message history and not by a new model call.

We can see in the call log that we now have only 2 model calls.

In [12]:
len(example_agent.log_calls)

2